<a href="https://colab.research.google.com/github/josefrancisco81788/Python-Projects/blob/main/Case_Gillete_Stadium_(Linear_Optimization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gillete Stadium

The New England Patriots are a professional American football team based in the Greater
Boston area. Their home games are played at Gillette Stadium in the town of
Foxborough, Massachusetts. The seating capacity of Gillette Stadium is 60,000 (not
including club and luxury seats). The Patriots play ten (10) games at Gillette Stadium
during the season and fans have two options to get tickets:
• Season Passes: Every season pass grants the holder access to two seats on every home
game for one season. Season passes are sold a few months before the start of the season.
Based on historical data, the forecasted demand for season passes is

DS = 20,000 - 5 PS,

where PS is the price of a pass and DS is the number of season passes that could be
sold.
• Game Tickets: A game ticket grants the holder access to one seat in a single specific
game. Game tickets are sold throughout the season before each game. For each game,
the forecasted demand for game tickets is

DG = 100,000 - 200 PG,

where PG is the price of a ticket and DG is the number of game tickets that could be
sold.

a) What are the optimal prices that should be charged for each type of ticket.
What are the revenues that the Patriots will make during the entire season? What is the marginal value of an extra unit of capacity (i.e., an extra seat on the stadium) over the entire season?

b) Do you see any problem in trying to implement the pricing strategy derived in
part (a)? If so, how would you tackle this problem? Explain. If needed, propose an alternative pricing scheme that is more acceptable to implement.

# Installation of Pyomo and Ipopt (via Conda)

In [ ]:
# Install Pyomo and necessary libraries
!pip install -q pyomo numpy scipy matplotlib

# Install conda (to manage dependencies)
!pip install -q condacolab
import condacolab
condacolab.install()

# Use conda to install Ipopt solver
!conda install -c conda-forge ipopt coincbc

# Add Ipopt executable to PATH (modify the path if needed)
import os
os.environ["PATH"] += os.pathsep + '/usr/local/bin'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - coincbc
    - ipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-mp-3.1.0              |    h2cc385e_1006         1.1 MB  conda-forge

# Import Libraries

In [2]:
# Import required libraries from Pyomo
from pyomo.environ import *

# Create a SolverFactory instance for Ipopt
opt = SolverFactory('ipopt')

# Check if Ipopt is available
assert opt.available(exception_flag=False), "Ipopt solver is not available."

# Model Creation

In [9]:
# Define inverse demand function
def inverse_demand(D, m, quantity):
    price = (D - quantity) / m
    return price

# Create the optimization model
model = ConcreteModel()

# Include dual solution in the output
model.dual = Suffix(direction=Suffix.IMPORT)

# Define decision variables
model.qS = Var(within=NonNegativeReals)  # Demand for Season Passes
model.qG = Var(within=NonNegativeReals)  # Demand for Game Tickets

# Constants for demand functions
Ds = 20000  # Number of season passes that could be sold.
Dg = 100000  # Number of game tickets that could be sold.
Ps = 5  # Price of a pass
Pg = 200  # Price of a ticket

# Define objective function (to maximize revenue)
model.obj = Objective(
    expr=model.qS * inverse_demand(Ds, Ps, model.qS) + model.qG * inverse_demand(Dg, Pg, model.qG),
    sense=maximize
)

# Capacity constraint (seating capacity of Gillette Stadium)
capacity_available = 60000
model.capacity = Constraint(expr=2 * model.qS + model.qG <= capacity_available)

# Solve the model
results = opt.solve(model, tee=False)

# Compute marginal revenues
marginal_rev_S = (Ds - 2 * value(model.qS)) / Ps
marginal_rev_G = (Dg - 2 * value(model.qG)) / Pg

# Output results
print("Preliminary Values:")
print("Ds Value:", Ds)
print("qS Value:", value(model.qS))
print("Ps Value:", Ps)
print()
print("Marginal Revenue Season Passes:", marginal_rev_S)
print("Dg Value:", Dg)
print("qG Value:", value(model.qG))
print("Pg Value:", Pg)
print("Marginal Revenue Game Tickets:", marginal_rev_G)
print("Capacity:", model.dual[model.capacity])
print()
print("SOLUTION:")
print('Optimal Revenue:', value(model.obj))
print('Optimal Quantity Season Passes:', value(model.qS))
print('Optimal Quantity Game Tickets:', value(model.qG))
print('Optimal Price Season Passes:', inverse_demand(Ds, Ps, value(model.qS)))
print('Optimal Price Game Tickets:', inverse_demand(Dg, Pg, value(model.qG)))
print()
print("MARGINAL REVENUES:")
print('MR(qS):', marginal_rev_S)
print('MR(qG):', marginal_rev_G)
print()
print('LAGRANGE MULTIPLIER:')
print('Capacity:', model.dual[model.capacity])


Preliminary Values:
Ds Value: 20000
qS Value: 9545.454549999966
Ps Value: 5

Marginal Revenue Season Passes: 181.81818000001368
Dg Value: 100000
qG Value: 40909.09099999897
Pg Value: 200
Marginal Revenue Game Tickets: 90.90909000001032
Capacity: 90.90909000001179

SOLUTION:
Optimal Revenue: 32045454.554545358
Optimal Quantity Season Passes: 9545.454549999966
Optimal Quantity Game Tickets: 40909.09099999897
Optimal Price Season Passes: 2090.909090000007
Optimal Price Game Tickets: 295.45454500000517

MARGINAL REVENUES:
MR(qS): 181.81818000001368
MR(qG): 90.90909000001032

LAGRANGE MULTIPLIER:
Capacity: 90.90909000001179


a) **Optimal Pricing Strategy and Revenue Analysis**

Optimal Prices and Revenue: The results suggest optimal prices of approximately USD 2,090.91 for a season pass and USD 295.45 for a game ticket. These prices maximize total revenue for the season at USD 32,045,454.55.

Capacity Utilization and Marginal Value:

Season Passes (qS): 9,545 passes sold, granting access to 19,090 seats (2 per pass per game).
Game Tickets (qG): 40,909 individual tickets sold.
Total Seats Sold:
19,090 + 40,909 = 60,000 seats, fully utilizing the stadium capacity for 10 games.

Marginal Value of Extra Capacity: The Lagrange multiplier (90.91) indicates the additional revenue generated by adding one more seat to the stadium. This figure helps gauge the profitability of future capacity expansions.

b) **Potential Issues and Recommendations**

Problem with Current Strategy: The current strategy sets uniform prices for both season passes and game tickets. However, demand elasticity might differ significantly between regular and high-demand games, leading to suboptimal pricing across different games. Fans may perceive the season pass price as too high relative to the game ticket, potentially affecting sales.

Alternative Pricing Scheme:

Dynamic Ticket Pricing: Adjust game ticket prices based on demand, opponent strength, day of the week, or time of purchase.

Tiered Season Pass Pricing: Offer different tiers of season passes (e.g., premium seating, early purchase discounts) to appeal to various customer segments.

Bundling Discounts: Provide bundled discounts for buying multiple game tickets or passes, encouraging higher upfront sales.

By refining the pricing strategy to reflect demand fluctuations and customer preferences, the Patriots can better optimize revenues and maintain strong customer satisfaction.